원문 : 
[pytorch로 구현하는 Transformer (Attention is All You Need)](https://cpm0722.github.io/pytorch-implementation/transformer)

paper: [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

## 개괄 구조

sentence input -> sentence output.

- 어떤 형태의 인풋이든, 어떤 형태의 아웃풋이든 정하기에 따라 다르게 쓸 수 있다. (동일 sentence 출력 / 번역 / 역방향 / ...)

인코더 & 디코더

- 인코더 : input sentence를 Context('문맥'을 함축한 하나의 벡터)로. 정보를 빠뜨리지 않고 압축하는게 목표.
- 디코더 : Context & Some sentence를 output sentence로.

In [ ]:
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F # softmax & log_softmax
import numpy as np

In [1]:
class prototype_Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(prototype_Transformer, self).__init__() # Python2의 문법이지만 범용성을 위해 흔히 이렇게 사용한다.
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x, z):
        c = self.encoder(x) # context를 내놓는 인코더
        y = self.decoder(z, c) # context와 어떤 sentence를 받아 output을 내놓는 디코더
        return y

In [2]:
"""
Encoder의 인풋은 문장 임베딩. 인코더 내부에는 Encoder Layer 여러개로 구성된다. 
    - 논문에서는 6개 Layer 사용
    - 각 Layer는 input에 대해 더 높은 차원(넓은 관점 or 추상적)의 context를 담는다.
Encoder Layer는 같은 shape을 사용해야하고, 결국 input과 context의 shape도 같다: 여러개의 Layer가 연속적으로 input-output으로 이어지기때문.
"""
import copy

class prototype_Encoder(nn.Module):
    def __init__(self, encoder_layer, n_layer):
        super(prototype_Encoder, self).__init__()
        self.layers = []
        for i in range(n_layer):
            self.layers.append(copy.deepcopy(encoder_layer))

    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out) # 순서대로 앞의 output을 다음 input으로.
        return out

"""
Encoder Layer는 다음으로 구성:
    - Multi-Head Attention Layer
    - Position-wise Feed-Forward Layer
"""
class prototype_EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
        super(prototype_EncoderLayer, self).__init__()
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer

    def forward(self, x):
        out = self.multi_head_attention_layer(x)
        out = self.position_wise_feed_forward_layer(out)
        return out

## Self-Attention?

Multi-Head Attention은 Self-Attention을 병렬로 수행하는 Layer. Self-Attention은 문장 내 두 토큰 사이의 연관성을 찾아내는 방법론이다. (Self: 같은 문장 내를 의미, 여기에서 다른 토큰 간 Attention)

### RNN과 비교한 Self-Attention

RNN : 이전 시점까지의 토큰에 대한 hidden state 내부에 이전 정보가 저장됨. 이전 정보를 통해 연관성을 찾는다는 개념 자체는 동일하나, Self-Attention의 장점은:

    1. i번째 hidden state를 계산하기 위해 i-1번째 hidden state 필요...-> 병렬처리 불가능. 하지만 Self-Attention은 모든 token 쌍 사이의 attention을 한 번의 Matrix 곱연산으로 구해내어 병렬처리 가능
    2. RNN은 시간이 지남(시퀀스 진행)에 따라 이전 토큰 정보가 희미해짐, 먼 거리 토큰의 연관성 정보가 제대로 반영되지 않음. 반면 Self-Attention은 각 토큰간 관계를 direct로 구해내므로 보다 명확하게 관계를 잡아낸다.

### Query, Key, Value

1. Query : 현재 시점의 토큰. **"Attention"이란 Query의 어탠션이다!**
2. key : attention을 구하고자 하는 대상 토큰
3. Value : attention을 구하고자 하는 대상 토큰(Key와 동일 토큰)

- 기본 로직 : Query에 가장 부합하는(=attention이 높은) -> Key&Value 대상 토큰을 찾는다.
    - 문장의 각각의 Query할 토큰에 대해, 해당 토큰이 문장 내의 다른 토큰(Key&Value)들과 갖는 Attention score를 구하는 것 : 
    - Query 토큰(1 X d<sub>k</sub>)과 Key.T와 내적 -> Query 토큰의 Attention score(1 X 1) -> Value에 곱합(1 X d<sub>k</sub>) -> Query 토큰의 Attention(1 X d<sub>k</sub>)
    - 위 과정을 문장 전체의 Query 토큰(n X d<sub>k</sub>)에 대해 수행한다 -> Query 토큰들의 Attention(n X d<sub>k</sub>)
- Key&Value의 차이점? Key와 Value가 가리키는 토큰은 같은 토큰(Query가 찾는 토큰)인데, 추후 계산을 위해 이렇게 두 가지 값이 존재한다.
- 세 개의 벡터는 각각 서로 다른 Fully-Connected Layer로 생성된다.(모두 같은 input&output dimension과 shape)
- dimension : d<sub>k</sub> (--> Key의 차원수라는 말인데, 굳이 Key에 의미부여한건 아니다. 그냥 다 같은 값인데 이름을 Key로 붙인 것)

#### 계산 방법

Query * Key 행렬곱 = Attention Score. 이를 스칼라 값인 차원수(d<sub>k</sub>)로 나눠준다(grad vanishing 방지)
<br>

**-> 각 토큰이 쿼리와 얼마나 attention을 갖는지(연관성을 갖는지)를 표현해준다**

실제 계산할 때는 Query에 대해 문장 내 모든 토큰에 대한 attention을 구해야 한다 : 행렬곱!<br>
**-> Key와 Value는 전체 문장인 Matrix가 된다(각 토큰 벡터를 갖는 문장 행렬)**

Query * Key인 Attention Score에 SoftMax를 취하고, Value와 다시 행렬곱 = Query's Attention

### QKV 자체는 어떻게 구하나?

임베딩된 input sentence에 3개의 각기 다른(shape은 같은) Fully-Connected layer를 통과시켜서 얻는다.
- input sentence(n X d<sub>embed</sub>)와 Query FC layer(d<sub>embed</sub> X d<sub>k</sub>) 곱연산 = Query(n X d<sub>k</sub>)
- input sentence(n X d<sub>embed</sub>)와 Key FC layer(d<sub>embed</sub> X d<sub>k</sub>) 곱연산 = Key(n X d<sub>k</sub>)
- input sentence(n X d<sub>embed</sub>)와 Value FC layer(d<sub>embed</sub> X d<sub>k</sub>) 곱연산 = Value(n X d<sub>k</sub>)


### Pad Masking

어탠션 계산 중 d<sub>k</sub> 스케일링과 SoftMax 적용 사이에 Masking을 수행한다.

트랜스포머에도 다른 시퀀스 모델들처럼 input 시퀀스를 같은 수로 맞춰주기 위한 패딩이 들어가는데, **아무 정보가 없는 패딩에 어탠션을 부여하면 안되므로 pad masking을 수행**한다.

계산 방법 : masking을 위한 Matrix를 곱한다 => **패딩 토큰에 해당하는 요소는 모두 -inf, 그 외에는 1**인 행렬.

In [ ]:
"""
argument "mask"는 학습 요청하는 외부에서 가져오는 마스킹이다.(mini batch size에 맞춰서 만들어서 전달)
"""
def prototype_calculate_attention(self, query, key, value, mask):
    # Q, K, V shape : (batch_size, seq_len, d_k)
    d_k = key.size(-1) # key의 차원

    # attention_score shape : (batch_size, seq_len, seq_len)
    attention_score = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k) # Q X K^T / sqrt(d_k)

    if mask is not None:
        attention_score = attention_score.masked_fill(mask == 0, -1e9) # masking

    # attention_prob shape : (batch_size, seq_len, seq_len)
    attention_prob = F.softmax(attention_score, dim = -1) # softmax

    # output shape : (batch_size, seq_len, d_k)
    out = torch.matmul(attention_prob, value) # attention_prob X V
    return out

### Multi-Head Attention Layer

이제 Self-Attention을 이해했으니 다음 단계인 Multi-Head Attention을 이해해보자. 논문에서는 **Scaled Dot-Product Attention**이라고 부른다. 트랜스포머는 Scaled Dot Attention을 각 Encoder Layer마다 여러번(<i>h</i>번) 수행하여 그 결과를 종합한다.
<br>
--> 여러 Attention을 잘 반영하기 위해서이다. 한 번만 진행한 Attention이라면 강력하게 연관된 토큰들만 엮여서, 문장 내 함축된 모든 정보를 담아내기에는 힘들다.

**d<sub>model</sub>** : Scaled Dot-Product Attention 연산의 핵심 개념. 
- 이제 Q,K,V에 대해 여러번(<i>h</i>번) 수행하므로 3*<i>h</i>개의 FC Layer가 필요하다. 
- 이것으로 d<sub>k</sub> X <i>h</i> 형태의 모델을 형성한다. (실제 어탠션 Matrix의 전체 shape은 max_seq_len * (d<sub>k</sub> X <i>h</i>))
- 효율적인 병렬연산을 위해 Q,K,V 벡터를 따로 생성(d<sub>embed</sub> X d<sub>k</sub> 행렬을 3*<i>h</i>개)하지 않고, 한번에 d<sub>embed</sub> X d<sub>model</sub>의 weight matrix를 갖는 3개의 FC Layer를 만들게 된다.

==> 즉, 결론만 놓고 보면 Self-Attention과의 차이는 차원수 d<sub>k</sub>를 d<sub>model</sub>로 단순 확장한 것. **정보의 양을 더 많이 담는 차원으로 Q,K,V의 차원을 확장한 것이다.**

==> 실제 계산은 아래 transform()에서 보이듯 reshape하여 d<sub>model</sub>을 <i>h</i>와 d<sub>k</sub>로 분해해 차원을 늘리고, transpose한다.

### 중간 점검 : 용어들 정리

- max_seq_len OR <i>n</i>
- Q, K, V : 각각 Query, Key, Value에 해당하는 벡터 또는 행렬. 아래는 batch_size 없을 때 크기 예시)
    - Q = Input sentence(<i>n</i> X d<sub>embed</sub>) X Query FC Layer(d<sub>embed</sub> X d<sub>k</sub>)
    - K = Input sentence(<i>n</i> X d<sub>embed</sub>) X Key FC Layer(d<sub>embed</sub> X d<sub>k</sub>)
    - V = Input sentence(<i>n</i> X d<sub>embed</sub>) X Value FC Layer(d<sub>embed</sub> X d<sub>k</sub>)
- pad masking : 패딩 정보 제거용 행렬. 크기는 Q X K<sup>T</sup>를 수행한 이후와 같은 <i>seq_len</i> X <i>seq_len</i>
- d<sub>k</sub> : Q,K,V의 차원 수
- d<sub>embed</sub> : 토큰의 embedding dimension. 사실상 임베딩을 거쳐 나오는 이 차원수는 d<sub>model</sub>과 같은 값
- <i>h</i> : 헤드 수. 어탠션 수행할 횟수
- d<sub>model</sub> : Multi-Head Attention의 전체 FC Layer에 입력할 차원 수. d<sub>k</sub> * <i>h</i>
    - Multi-Head Attention의 FC Layer shape : d<sub>model</sub> X d<sub>embed</sub>

### 중간 점검 : 아케텍처 업데이트

위 기본 prototype의 트랜스포머 구조들을 padding, multi-head를 반영해 수정한다.

In [ ]:
import math

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, d_model, h, qkv_fc_layer, fc_layer):
        # qkv_fc_layer shape : (d_embed, d_model)
        # fc_layer shape : (d_model, d_embed)
        super(MultiHeadAttentionLayer, self).__init__()
        self.d_model = d_model
        self.h = h
        self.query_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.key_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.value_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.fc_layer = fc_layer

    # !!Transformer의 핵심인 forward 함수!!
    def forward(self, query, key, value, mask=None):
        # query, key, value 인자가 실제로 실제 Q,K,V 행렬이 아니라, 각각의 FC Layer로 입력할 input sentence
        # query, key, value shape : (batch_size, seq_len, d_embed)
        # mask shape : (batch_size, seq_len, seq_len)
        batch_size = query.shape[0]

        # x shape : (batch_size, seq_len, d_embed)
        # reshape to (batch_size, seq_len, h, d_k)
        def transform(x, fc_layer): # Q,K,V 행렬 구하기
            out = fc_layer(x) # (batch_size, seq_len, d_model)
            out = out.view(batch_size, -1, self.h, self.d_model//self.h) # (batch_size, seq_len, h, d_k)
            out = out.transpose(1, 2) # (batch_size, h, seq_len, d_k)
            return out # 이렇게 형태를 변형하는 이유는 아래 calculate_attention 함수에서 사용하기 위해서.

        # query, key, value shape after transform : (batch_size, h, seq_len, d_k)
        query = transform(query, self.query_fc_layer)
        key = transform(key, self.key_fc_layer)
        value = transform(value, self.value_fc_layer)

        if mask is not None:
            # 아래 calculate_attention에서 마스킹을 브로드캐스팅으로 수행하기 위해 변형해준다.
            mask = mask.unsqueeze(1) # (batch_size, 1, seq_len, seq_len)

        out = self.calculate_attention(query, key, value, mask) # out shape : (batch_size, h, seq_len, d_k)
        out = out.transpose(1, 2) # (batch_size, seq_len, h, d_k)
        """
        contiguous : 연속적인 메모리 텐서를 할당해주는 함수로, 
        차원을 변형할때 실제론 메타정보만 바뀌는 view 같은 함수에 활용하면 다음 메모리로 넘어가는 
        stride가 고정되지 않고 잘 할당된다는 것 같다.
        """
        out = out.contiguous().view(batch_size, -1, self.d_model) # (batch_size, seq_len, d_model)
        out = self.fc_layer(out) # (batch_size, seq_len, d_embed), 인풋 형태로 복귀

        return out

    """
    Self-Attention때와 똑같은 함수지만, 해당 함수의 input & output shape이 다름에 주의.
        -> h에 의해 차원수가 하나 늘어난 정도로, 연산 자체는 브로드캐스팅을 활용하면 완전히 똑같다.
    (*참고) matmul의 연산방식 : "만약 배열이 2차원보다 클 경우, 마지막 2개의 축으로 이루어진 행렬을 나머지 축에 따라 쌓아놓은 것이라고 간주한다."
        즉, (1) 맨 뒤 두 개 차원으로 행렬곱이 가능한 경우 & (2) 맨 뒤 두 개 외에 나머지 차원축 상의 갯수가 같은 경우
    """
    def calculate_attention(self, query, key, value, mask):
        # Self-Attention에서의 Q, K, V shape : (batch_size, seq_len, d_k)
        # 여기에서의 Q, K, V shape : (batch_size, h, seq_len, d_k)
        d_k = key.size(-1)
        
        # attention_score shape : (batch_size, seq_len, seq_len)
        # transpose(-2,-1) : dimension -2와 dimension -1을 swapping. 즉, 전치를 위해 seq_len(n)과 d_k를 바꾸기.
        attention_score = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k) # Q X K^T / sqrt(d_k)
        if mask is not None:
            attention_score = attention_score.masked_fill(mask == 0, -1e9)
        attention_prob = F.softmax(attention_score, dim = -1) # softmax
        out = torch.matmul(attention_prob, value) # attention_prob X V

        return out # out shape : (batch_size, h, seq_len, d_k)

In [ ]:
"""
masking 추가 : 학습 요청하는 외부에서 들어오는 mask 인자(mini-batch 사이즈에 맞춘 마스킹)
"""

class prototype_Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(prototype_Transformer, self).__init__() # Python2의 문법이지만 범용성을 위해 흔히 이렇게 사용한다.
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, mask):
        encoder_output = self.encoder(source, mask) # context를 내놓는 인코더
        out = self.decoder(target, encoder_output) # context와 어떤 sentence를 받아 output을 내놓는 디코더
        return out


class Encoder(nn.Module):
    def __init__(self, encoder_layer, n_layer):
        super(Encoder, self).__init__()
        self.layers = []
        for i in range(n_layer):
            self.layers.append(copy.deepcopy(encoder_layer))

    def forward(self, x, mask): # mask 추가
        out = x
        for layer in self.layers:
            out = layer(out, mask)
        return out


class prototype_EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
        super(prototype_EncoderLayer, self).__init__()
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer

    def forward(self, x, mask): # mask 추가
        out = self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask) # multi-head attention 클래스에 맞게 args 조정
        out = self.position_wise_feed_forward_layer(out)
        return out


### Position-wise FFW

단순하게 2개 FC layer를 갖는 구조. 각각 아래 shape의 weight matrix를 가지는데,
- d<sub>embed</sub> X d<sub>ff</sub> (+ ReLU)
- d<sub>ff</sub> X d<sub>embed</sub>

즉, Position-wise FFW를 거치면 input shape을 그대로 유지한다.(=다음의 Encoder Layer에 그대로 넘겨주기)
<br>위 내용을 논문의 아래 식으로 표현할 수 있다.

$$FFN(x) = max(0,xW_1 + b_1)W_2 + b_2$$

In [ ]:
class PositionWiseFeedForwardLayer(nn.Module):
	def __init__(self, first_fc_layer, second_fc_layer):
		self.first_fc_layer = first_fc_layer
		self.second_fc_layer = second_fc_layer
	
	def forward(self, x):
		out = self.first_fc_layer(x)
		out = F.relu(out)
		out = self.dropout(out)
		out = self.second_fc_layer(out)
		return out

### Norm Layer(Residual Connection)

Encoder Layer는 Multi-Head attention Layer & Position-wise FFW 두 종류의 Layer로 구성된다.
<br>이 두 가지의 Layer는 ***Residual Connection***으로 둘러쌓여 있다 : 
- Residual Connection? : $y=f(x)$를 $y=f(x)+x$ 로 변형
- 인풋을 추가로 더한 값을 쓰므로, back prop 중 발생하는 gradient vanishing을 방지한다.

-> 여기에 추가적으로 논문에서 채택한 Layer Normalization까지 추가.

In [ ]:
class ResidualConnectionLayer(nn.Module):
	def __init__(self, norm_layer):
		super(ResidualConnectionLayer, self).__init__()
		self.norm_layer = norm_layer

	def forward(self, x, sub_layer):
		out = sub_layer(x) + x
		out = self.norm_layer(out)
		return out

-> 다시 Encoder Layer를 업데이트 해주자.

In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer, norm_layer):
        super(EncoderLayer, self).__init__()
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
        self.residual_connection_layers = [ResidualConnectionLayer(copy.deepcopy(norm_layer)) for i in range(2)]

    """
    이전 multi-head + position-wise FFW 두 단의 레이어를 Residual conn layer로 감싸주자.
    """
    def forward(self, x, mask):
        out = self.residual_connection_layers[0](x, lambda x: self.multi_head_attention_layer(x, x, x, mask))
        out = self.residual_connection_layers[1](x, lambda x: self.position_wise_feed_forward_layer(x))
        return out

--------

## Decoder

기본적으로는 Encoder 구조와 비슷해서 layer를 그대로 가져다 쓰고, 몇 가지 변경을 가해주면 된다.

우선 Decoder의 I/O부터 확실하게 정의하자:
- inputs : Context(Encoder output) | Some Sentence
    - Context로 인풋의 텍스트의 정보를 함축한 것을 받는다. 이때 Encoder는 input과 같은 shape의 output을 내놓는다
    - Some Sentence - ***"Teacher Forcing"*** : RNN이던 Transformer던 어떤 NLP 관련 모델을 최초로 학습할 때, 과거정보 or 위치정보를 다음 cell 학습에 쓰기 때문에 완전 랜덤의 최초 init weight으로는 다음 학습이 계속 엉터리가 될 가능성이 크다. 따라서 ground truth를 input으로 쓴다는 개념을 teacher forcing이라고 한다. 대부분의 NLP 모델에서 거의 항상 사용되는 기법.
- output : Output Sentence

구조 자체는 Encoder와 거의 똑같고, 다만 각 Decoder Layer마다 Encoder의 Context가 인풋으로 들어간다는 점만 차이가 난다.

#### Transformer에서의 Teacher Forcing
RNN의 경우 순차적인 연산으로 인해 input으로 ground truth를 거의 그대로 사용했지만, Transformer에서는 이처럼 임베딩된 값을 그대로 input으로 넣어버리게 된다면 self-attention으로 생성해내야 하는 정답 token을 그대로 주는 것이라 안 된다.

=> 따라서 masking을 적용한다 :~~$i$번째 token에게 1 ~ (i-1) token은 보이지 않는다.~~
<br>=> 해당 블로그에서 잘못 써놓은 듯 하다. 코드 구현 내용처럼, 위 내용과 반대로 1 ~ $(i-1)$까지의 token만 바라보고 나머지를 masking out 한다.

=> **"subsequent masking"**

In [ ]:

"""
numpy.triu(arr, k) : 상삼각(upper triangular)행렬에서 해당 원소만 남기게 해준다. 
- 맨 위 row부터 한줄한줄 내려갈때마다 각 row의 요소들이 앞에서부터 0으로 채워진다.
- k :오프셋이다. 맨 윗줄부터 몇 개의 0으로 시작할지 의미. default는 0으로, k=0이면 맨 윗 row에는 0이 안생긴다.
"""
def subsequent_mask(size): # size에는 seq_len이 들어옴
	atten_shape = (1, size, size) # attention shape = (1, seq_len, seq_len) 여기서 batch size는 고려 안함
	mask = np.triu(np.ones(atten_shape), k=1).astype('uint8') # masking with upper triangle matrix
	# 위에서 만든 상삼각행렬을 아래에서 하삼각행렬로 바꿔준다.
	return torch.from_numpy(mask)==0 # reverse (masking=False, non-masking=True)


"""
subsequent mask를 생성하고 pad mask와 결합한다.
Transformer 내부가 아닌, 외부의 batch 처리 중에 수행된다. (masking 과정은 전처리 과정이다.)
"""
def make_std_mask(target, pad):
	target_mask = (target != pad) # pad masking
	target_mask = target_mask.unsqueeze(-2) # reshape (n_batch, seq_len) -> (n_batch, 1, seq_len)
	target_mask = target_mask & Tensor(subsequent_mask(target.size(-1)).type_as(target_mask.data)) # pad_masking & subsequent_masking
	return target_mask

-> Transformer 전체 구조에서 Decoder input을 업데이트하자.

In [ ]:
class Transformer(nn.Module):

	def __init__(self, encoder, decoder):
		super(Transformer, self).__init__()
		self.encoder = encoder
		self.decoder = decoder

	# 디코더의 마스킹을 구분하여 넣어주자.
	def forward(self, source, target, source_mask, target_mask):
		encoder_output = self.encoder(source, source_mask)
		out = self.decoder(target, target_mask, encoder_output)
		return out

-------

## Inputs
### Positional Encoding

기본적인 단순 embedding에 더해, Transformer의 embedding은 PositionalEncoding이 더해진다
- 즉, token embedding + PositionalEncoding의 Sequential 모델이라 할 수 있다

In [ ]:
class TransformerEmbedding(nn.Module):
    """
    단순 embedding + PositionalEncoding 전체 연결을 정의
    """
    def __init__(self, embedding, positional_encoding):
        super(TransformerEmbedding, self).__init__(): # 상속 클래스 init
        self.embedding = nn.Sequential(embedding, positional_encoding) # 기존 embedding + PositionalEncoding

    def forward(self, x):
        out = self.embedding(x)
        return out

class Embedding(nn.Module):
    """
    단순 embedding
    """
    def __init__(self, d_embed, vocab):
        super().__init__()
        self.embedding = nn.Embedding(len(vocab), d_embed)
        self.vocab = vocab
        self.d_embed = d_embed
    def forward(self, x):
        out = self.embedding(x) * math.sqrt(self.d_embed) # scaling : PositionalEncoding의 영향을 상대적으로 적게 가져가기 위해 단순 임베딩의 크기를 늘린다.
        return out

**PositionalEncoding 클래스**

PositionalEncoding의 목적 : 위치 정보(=토큰 순서 =index num)를 정규화시키기
- 단순히 index num으로 쓰게 된다면? : 새로운 test input의 길이가 positional encoding이 본 적 없는 긴 길이일때 문제가 생긴다..
- 따라서 sin & cos 함수를 써서 [-1,1] 안으로 제한하는 것

아래 수식은 논문 수식과 약간은 차이가 있는거 같은데(자연상수 e를 밑으로 하는 div_term과 달리 그냥 10을 밑으로 함?)
아마 큰 차이는 없는거 아닐까?

수식 div_term : $$e^{-\log 10000 \times 2i/d_{embed}}=e^{-\log 10000} \cdot e^{2i/d_{embed}}=$$
$$ {1 \over 10000} \cdot e^{2i/d_{embed}}$$
수식 (1) (짝수) : $$ \sin ({{pos \times e^{2i/d_{embed}}} \over 10000})$$
수식 (2) (홀수) : $$ \cos ({{pos \times e^{2i/d_{embed}}} \over 10000})$$

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_embed, max_seq_len=5000):
        super().__init__()
        encoding = torch.zeros(max_seq_len, d_embed) # 임베딩 최종 결과물 shape인 (n, d_embed)
        position = torch.arange(0, max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_embed, 2) * -(math.log(10000.0) / d_embed)) # 수식 div_term
        encoding[:, 0::2] = torch.sin(position * div_term) # 수식 (1)
        encoding[:, 1::2] = torch.cos(position * div_term) # 수식 (2)
        self.encoding = encoding

    def forward(self, x):
        out = x + Variable(self.encoding[:, :x.size(1)], requires_grad=False) # 여기 forward에서 Variable이 학습되지 않도록 한다.

-------